In [234]:
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)

test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)

In [235]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [236]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [237]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [238]:
test_data.data.shape

torch.Size([10000, 28, 28])

In [239]:
from torch.utils.data import DataLoader

loaders = {
    'train': DataLoader(train_data, batch_size=100, shuffle=True, num_workers=1),
    'test': DataLoader(test_data, batch_size=100, shuffle=True, num_workers=1)
}

In [240]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5, stride=1)
        self.conv2_dropout = nn.Dropout2d()

        self.fc1 = nn.Linear(in_features=20*4*4, out_features=50)
        self.fc2 = nn.Linear(in_features=50, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2)

        x = F.relu(self.conv2_dropout(self.conv2(x)))
        x = F.max_pool2d(x, kernel_size=2)

        x = x.view(-1, 20*4*4)

        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        
        return F.softmax(x)



In [241]:
model1 = CNN()
model1.parameters()

<generator object Module.parameters at 0x160e3e5e0>

In [242]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model1.parameters(), lr=0.001)

In [243]:
epochs = 1000

def train(epoch):
    model1.train()

    for batch_idx, (data, target) in enumerate(loaders['train']):
        optimizer.zero_grad()

        output = model1(data)

        loss = loss_fn(output, target)
        loss.backward()
        
        optimizer.step()

        if batch_idx % 20 == 0:
            print(f"Epoch: {epoch} | [{batch_idx * len(data)} / {len(loaders['train'].dataset)}] | Loss: {loss}")

In [244]:
def test():
    model1.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in loaders['test']:
            output = model1(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(loaders['test'].dataset)
    print(f"Average Loss: {test_loss} | Correct: {correct} / 10000")

In [245]:
for epoch in range(1, 11):
    train(epoch)
    test()

/var/folders/y_/g1zfkg_x123fctpz09drl4h00000gn/T/ipykernel_3189/2886317348.py:31: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Epoch: 1 | [0 / 60000] | Loss: 2.304447889328003
Epoch: 1 | [2000 / 60000] | Loss: 2.291552782058716
Epoch: 1 | [4000 / 60000] | Loss: 2.1775052547454834
Epoch: 1 | [6000 / 60000] | Loss: 2.0189530849456787
Epoch: 1 | [8000 / 60000] | Loss: 1.9247615337371826
Epoch: 1 | [10000 / 60000] | Loss: 1.8670361042022705
Epoch: 1 | [12000 / 60000] | Loss: 1.872247576713562
Epoch: 1 | [14000 / 60000] | Loss: 1.7582632303237915
Epoch: 1 | [16000 / 60000] | Loss: 1.748719334602356
Epoch: 1 | [18000 / 60000] | Loss: 1.6514629125595093
Epoch: 1 | [20000 / 60000] | Loss: 1.6942447423934937
Epoch: 1 | [22000 / 60000] | Loss: 1.7052814960479736
Epoch: 1 | [24000 / 60000] | Loss: 1.6725753545761108
Epoch: 1 | [26000 / 60000] | Loss: 1.6708776950836182
Epoch: 1 | [28000 / 60000] | Loss: 1.618985652923584
Epoch: 1 | [30000 / 60000] | Loss: 1.714369773864746
Epoch: 1 | [32000 / 60000] | Loss: 1.6654967069625854
Epoch: 1 | [34000 / 60000] | Loss: 1.6816015243530273
Epoch: 1 | [36000 / 60000] | Loss: 1.62625

In [246]:
from pathlib import Path

MODEL_PATH = Path("Models")
MODEL_PATH.mkdir(parents=1, exist_ok=1)

MODEL_NAME = "01_model_0.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model1.state_dict(), f=MODEL_SAVE_PATH)

Saving model to: Models/01_model_0.pth
